*problem: GLIBCXX 3.4.30 not found in conda environment*

solution: 
check:
strings /usr/lib/x86_64-linux-gnu/libstdc++.so.6 | grep GLIBCXX
then change:
ln -sf /usr/lib/x86_64-linux-gnu/libstdc++.so.6 /home/anavani/anaconda3/envs/dmcgb/bin/../lib/libstdc++.so.6

* Convert delta actions to absolute actions
```bash
python utils/robomimic_convert_action.py --dataset=robomimic/datasets/lift/ph/low_dim.hdf5
```

* Save waypoints
```bash
python utils/robomimic_save_waypoints.py --dataset=robomimic/datasets/lift/ph/low_dim.hdf5 --err_threshold=0.005
```

* Replay waypoints (save 3 videos and 3D visualizations by default)
```bash
mkdir video
python example/robomimic_waypoint_replay.py --dataset=robomimic/datasets/lift/ph/low_dim.hdf5 \
    --record_video --video_path video/lift_waypoint.mp4 --task=lift \
    --plot_3d --auto_waypoint --err_threshold=0.005

* Train DP
python diffusion_policy/train.py --config-dir=config --config-name=waypoint_image_lift_ph_diffusion_policy_transformer.yaml hydra.run.dir='data/outputs/${now:%Y.%m.%d}/${now:%H.%M.%S}_${name}_${task_name}'

In [ ]:
import mujoco_py